#### Llama2 f16 safetensors 모델 
1. 과정
   * [12]과정을 통해 Meta Llama2 모델을 Llama2-hf 모델로 변환한다.
   * [14]과정을 통해 Llama2-hf 모델을 safetensors(Tensorflow 저장방식) 방식으로 저장한다.
   * model = AutoModelForCausalLM.from_pretrained(model_id) 이 방식의 모델로딩에는 문제가 있음.
   * [55]Generate_Koalpaca_Safetensors 방식으로 로딩해야 정상작동함.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:

model_id = "/data/bwllm/models/koalpaca-polyglot-12.8bsf"
#model_id = "/data/bwllm/models/casllm-base-7b-sf"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# 모델의 첫 번째 파라미터의 데이터 타입 확인
first_param_dtype = next(model.parameters()).dtype
print(f"First parameter data type: {first_param_dtype}")

In [ ]:
# 모델의 모든 파라미터의 데이터 타입 확인
for name, param in model.named_parameters():
    print(f"{name} data type: {param.dtype}")

In [ ]:
# 양자화 여부 확인
if 'quantized' in str(first_param_dtype):
    print("The model is quantized.")
else:
    print("The model is not quantized.")

In [ ]:
# 모델을 float16으로 변환
model = model.half()

In [ ]:
# 모델의 모든 파라미터의 데이터 타입 확인
for name, param in model.named_parameters():
    print(f"{name} data type: {param.dtype}")

In [ ]:

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

# 장치 확인   
print(device)

# 모델을 해당 디바이스로 이동
model.to(device)

In [ ]:
print(model)

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=model_id,
    #device=0,
)

In [ ]:
def make_answer(prompt_status, question):
    
    messages = prompt_status + [ {"role": "질문", "content": question}]    
    prompt_text = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )
    return prompt_text

In [ ]:
# 문장 생성
def generate(prompt_text):
    answer = pipe(
        prompt_text,
        do_sample=True,
        max_new_tokens=2000,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    return answer[0]['generated_text']

In [ ]:
prompt_status = [
            {
                "role": "맥락",
                "content": "KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.",
            },
        ]

prompt_status2 = []

question = "정보화시스템에서의 감리절차에 대해 알려줘"
prompt_text = make_answer(prompt_status2, question)

print(prompt_text)

In [ ]:
# 답변 텍스트만 출력
answer = generate(prompt_text)
print(answer)